In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
data = pd.read_csv(url)

# Display first few rows of the dataset
print(data.head())

# Data Cleaning and Handling Missing Values
# Drop columns that are not useful for prediction
data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

# Fill missing values
# Fill 'Age' with the median value
data['Age'].fillna(data['Age'].median(), inplace=True)

# Fill 'Embarked' with the most common value
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

# Drop rows where 'Fare' is missing (if any)
data.dropna(subset=['Fare'], inplace=True)

# Feature Engineering
# Create new feature 'FamilySize' from 'SibSp' and 'Parch'
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

# Create new feature 'IsAlone' from 'FamilySize'
data['IsAlone'] = np.where(data['FamilySize'] == 1, 1, 0)

# Transforming categorical variables to numerical
# Define categorical features to encode
categorical_features = ['Sex', 'Embarked']

# Define numerical features to scale
numerical_features = ['Age', 'Fare', 'FamilySize']

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Separate features and target variable
X = data.drop(columns=['Survived'])
y = data['Survived']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the preprocessing pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Fit and transform the training data
X_train_preprocessed = pipeline.fit_transform(X_train)

# Transform the testing data
X_test_preprocessed = pipeline.transform(X_test)

# Convert the preprocessed data back to a DataFrame
preprocessed_columns = (
    numerical_features +
    list(pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features))
)
X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed, columns=preprocessed_columns)
X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, columns=preprocessed_columns)

# Display the first few rows of the preprocessed training data
print(X_train_preprocessed_df.head())


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
  